In [1]:
from selenium import webdriver # Lấy dữ liệu từ trang web bằng việc điều khiển trình duyệt ảo
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime, timedelta
from time import sleep
import time
import pandas as pd
import re

# Khởi tạo driver

In [2]:
# Thiết lập tùy chọn cho trình duyệt Chrome
options = webdriver.ChromeOptions()

# Thêm các tùy chọn cho trình duyệt
options.add_argument('--window-size=860,1080') #Định kích thước cửa sổ theo chiều rộng, cao.
options.add_argument("--no-sandbox")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-extensions")
options.add_argument("--incognito") #Mở trang ẩn danh, không lưu trữ dữ liệu
options.add_argument("--disable-dev-shm-usage")  # Khắc phục vấn đề hạn chế tài nguyên
options.add_argument("--disable-infobars")  # Ẩn thông báo "Chrome đang được kiểm soát bởi phần mềm tự động"
options.add_argument('--ignore-certificate-errors') #Bỏ qua lỗi chứng chỉ SSL/TLS, cho phép truy cập các trang sử dụng chứng chỉ không hợp lệ.
options.add_argument('--disable-notifications') #Tắt thông báo pop-up.
options.add_argument('--disable-infobars') #Tắt thanh thông tin.


# Hàm xử lý các biến class

In [3]:
def remove_class(selector):
    splits = selector.split(" ")
    for i, split in enumerate(splits):
        splits[i] = split.split(".")[0]
    return " ".join(splits)

In [14]:
selector = "#mount_0_0_wx > div > div:nth-child(1) > div > div.x9f619.x1n2onr6.x1ja2u2z > div > div > div.x78zum5.xdt5ytf.x1t2pt76.x1n2onr6.x1ja2u2z.x10cihs4 > div.x78zum5.xdt5ytf.x1iyjqo2.x1us19tq > div > div.x9f619.x2lah0s.x1n2onr6.x78zum5.x1iyjqo2.x1t2pt76.x1lspesw > div > div > div.x78zum5.xdt5ytf.x1iyjqo2 > div > div > div.x6s0dn4.x78zum5.xdt5ytf.xwib8y2.xh8yej3 > div > div > div > div:nth-child(2) > div:nth-child(2) > div:nth-child({i}) > div > div > div > div > div > div > div > div > div > div:nth-child(13) > div > div"
a = remove_class(selector)
a

'#mount_0_0_wx > div > div:nth-child(1) > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div:nth-child(2) > div:nth-child(2) > div:nth-child({i}) > div > div > div > div > div > div > div > div > div > div:nth-child(13) > div > div'

In [4]:
#2. Khởi tạo WebDriver cho Chrome với các tùy chọn đã được thiết lập trong options
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
                        #Tự động cập nhập trình điều khiển Chrome mới nhất
#3. Đăng nhập và truy cập vào trang/ group
#3.1. Vào trang/ group
url = 'https://www.facebook.com/groups/utcshop0924858888?locale=vi_VN'
driver.get(url)
wait = WebDriverWait(driver, 5)


# Login vào facebook

In [5]:
# txtuser = driver.find_element(By.CSS_SELECTOR, '#email')
# txtuser.send_keys("tunleanh4112003@gmail.com") # điền user

# txtpass = driver.find_element(By.CSS_SELECTOR,'#pass')
# txtpass.send_keys("Tunfacebook4112003") # điền pass

# login_in_selector = "/html/body/div[1]/div[1]/div[1]/div/div/div/div[2]/div/div[1]/form/div[2]/button"
# login_in_button = driver.find_element(By.XPATH,login_in_selector)
# login_in_button.click() # click vào nút đăng nhập

#3.2. Điền thông tin vào ô user và pass
txtuser = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#login_popup_cta_form > div > div > div > label > div > div > input'))) # chờ đến khi fb hiện element (hạn chế dùng XPATH vì hay thay đổi)
txtuser.send_keys("tunleanh4112003@gmail.com") # điền user

txtpass = driver.find_element(By.CSS_SELECTOR,'#login_popup_cta_form > div > div:nth-child(4) > div > label > div > div > input')
txtpass.send_keys("Tunfacebook4112003") # điền pass

login_in_selector = "#login_popup_cta_form > div > div:nth-child(5) > div"
login_in_button = driver.find_element(By.CSS_SELECTOR,login_in_selector)
login_in_button.click() # click vào nút đăng nhập


In [6]:
class_list = ["#mount_0_0_YA"]
#mount_0_0_YA > div > div:nth-child(1) > div > div:nth-child(3) > div.x9f619.x1s65kcs.x16xn7b0.xixxii4.x17qophe.x13vifvy.xj35x94.xhtitgo.xkreb8t > div > div > div > div > div > label

# Chọn những bài viết mới

In [9]:
# Xác định CSS selector cho button và post mới
button_new_selector = f"{class_list[0]} > div > div:nth-child(1) > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div:nth-child(2) > div:nth-child(2) > div > div > div > div > div > div > div > div > div > span > div > div > div > div:nth-child(2) > div > svg"
posts_new_selector = f"{class_list[0]} > div > div:nth-child(1) > div > div > div > div > div:nth-child(2) > div > div > div > div > div > div > div > div > div > div > div > div > div > div:nth-child(3) > div"

# Tìm và click vào button mới
button_new = driver.find_element(By.CSS_SELECTOR, button_new_selector)
button_new.click()

# Tìm và click vào post mới
posts_new = driver.find_element(By.CSS_SELECTOR, posts_new_selector)
posts_new.click()


# Nhập từ khóa tìm kiếm

In [13]:
# keywork_in_selector = f"{class_list[0]} > div > div:nth-child(1) > div > div:nth-child(3) > div > div > div > div > div > div > label"
# keywork_in_button = driver.find_element(By.CSS_SELECTOR,keywork_in_selector)
# keywork_in_button.click() # click vào tìm kiếm

# keywork = driver.find_element(By.CSS_SELECTOR, f"{class_list[0]} > div > div:nth-child(1) > div > div:nth-child(3) > div > div > div > div > div > div > label > input")
# keywork.send_keys("review iphone 15") # điền keyword

# keywork.send_keys(Keys.ENTER)

# Chọn mục bài viết

In [15]:
# post_in_selector = f"{class_list[0]} > div > div:nth-child(1) > div > div > div > div > div > div > div > div > div > div > div > div > div > div:nth-child(2) > div > div:nth-child(2) > div > a > div > div > div"
# post_in_button = driver.find_element(By.CSS_SELECTOR,post_in_selector)
# post_in_button.click() # click vào tìm kiếm

In [12]:
def parse_time(text):
    # Sử dụng regular expressions để tìm số và đơn vị thời gian
    match = re.search(r'(\d+)\s*(phút|giờ|ngày)', text)
    if not match:
        return None
    
    amount, unit = match.groups()
    amount = int(amount)
    
    if 'phút' in unit:
        return datetime.timedelta(minutes=amount)
    elif 'giờ' in unit:
        return datetime.timedelta(hours=amount)
    elif 'ngày' in unit:
        return datetime.timedelta(days=amount)
    else:
        return None

In [11]:
# Khởi tạo DataFrame
df_posts = pd.DataFrame(columns=['time', 'num_comment', 'like_post', 'content_post'])
for i in range(2, 200):
    try:
        list_id = []  # Khởi tạo lại list trống mỗi lần lặp qua một phần tử mới
        posts_data = []  # Reset list tạm để lưu dữ liệu cho mỗi phần tử i
        try:
            # Cuộn đến phần tử sử dụng CSS selector
            scroll_to_selector = f"{class_list[0]} > div > div:nth-child(1) > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div:nth-child(2) > div:nth-child(2) > div:nth-child({i}) > div > div > div > div > div > div > div > div > div > div:nth-child(13) > div > div"
            element_to_scroll_to = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, scroll_to_selector)))
            driver.execute_script("arguments[0].scrollIntoView(true);", element_to_scroll_to)

            # Xây dựng một XPath để chọn tất cả các phần tử <a>, <div>, <h3>, và <span> con của phần tử mục tiêu
            child_elements_xpath = ".//*[self::a or self::div or self::h3 or self::span]"
            child_elements = element_to_scroll_to.find_elements(By.XPATH, child_elements_xpath)

            # Lặp qua từng phần tử con và lưu 'id' vào list_id
            for element in child_elements:
                element_id = element.get_attribute('id')
                if element_id:  # Kiểm tra xem phần tử có 'id' không trước khi thêm vào list
                    # Thêm \\ vào trước và sau dấu hai chấm
                    formatted_id = f"\\{element_id}\\"
                    formatted_id = formatted_id.replace(":", "\\:")
                    # Loại bỏ \\ ở đầu và cuối chuỗi
                    formatted_id_corrected = formatted_id[1:-1]  # Cắt bỏ ký tự đầu và cuối
                    list_id.append(formatted_id_corrected)
            # Ở đây bạn có thể sử dụng list_id cho mục đích cụ thể của phần tử hiện tại
            print(list_id)

        except TimeoutException:
            print(f"Không tìm thấy phần tử hoặc không thể lấy id cho phần tử {i}")

        content_found = False
    # Lặp qua các phần tử trong list_id
        for id in list_id:
            content = None
            num_comment = None
            time_text = None
            emojis = None
            clicked = False  # Biến kiểm tra xem đã click vào nút "Xem thêm" thành công hay không

            # Cố gắng tìm và click vào nút "Xem thêm" nếu có
            try:
                click_xemthem_selector = f"#{id} > div > div > span > div > div:nth-child(10) > div"
                click_xemthem = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, click_xemthem_selector)))
                driver.execute_script("arguments[0].click();", click_xemthem)
                clicked = True
            except TimeoutException:
                pass  # Nếu không tìm thấy nút "Xem thêm", tiếp tục

            # Cố gắng lấy nội dung sử dụng selector đầu tiên
            try:
                content_selector_1 = f"#{id} > div > div > span"
                content_element = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, content_selector_1)))
                content = content_element.text.strip()
            except TimeoutException:
                # Nếu không tìm thấy nội dung với selector đầu tiên, thử với selector thứ hai
                try:
                    content_selector_2 = f"#{id} > div"
                    content_element = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, content_selector_2)))
                    content = content_element.text.strip()
                except TimeoutException:
                    pass

            # Lấy thời gian bài post
            try:
                time_selector = f"#{id} > span:nth-child(4) > span > a > span"
                time_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, time_selector)))
                time_text = time_element.text  # Ví dụ "59 phút"    
            except TimeoutException:
                pass

            # Lấy số comment và cảm xúc của bài post
            try:
                comment_selector = f"{class_list[0]} > div > div:nth-child(1) > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div:nth-child(2) > div:nth-child(2) > div:nth-child({i}) > div > div > div > div > div > div > div > div > div > div:nth-child(13) > div > div > div:nth-child(4) > div > div > div:nth-child(1) > div > div"
                # Lấy phần tử dựa trên comment_selector
                comment_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, comment_selector)))

                # Lấy toàn bộ text từ comment_element
                num_comment = comment_element.text

                # Tìm tất cả các thẻ <div> con có thuộc tính aria-label bên trong comment_element
                emoji_elements = comment_element.find_elements(By.XPATH, ".//div[@aria-label]")

                # Lặp qua từng phần tử emoji để lấy giá trị aria-label và lưu vào list
                emoji_list = [element.get_attribute('aria-label') for element in emoji_elements]

                # Nối các giá trị aria-label lại với nhau, mỗi giá trị cách nhau bởi dấu phẩy
                emojis = ', '.join(emoji_list)

            except Exception as e:
                num_comment = "Error"
                emojis = "Error"
                print(f"An error occurred: {str(e)}")  # In ra lỗi nếu có

            # Thêm thông tin vào list tạm nếu có nội dung
            if content is not None:
                post_info = {
                    'time': time_text,
                    'num_comment': num_comment,
                    'like_post': emojis,
                    'content_post': content.replace("\n", "\\n")  # Thay thế xuống dòng
                }
                posts_data.append(post_info)
                print(posts_data)
        # Sau khi lặp qua tất cả id, thêm dữ liệu từ list tạm vào DataFrame
        if posts_data:  # Kiểm tra nếu list không trống
            df_posts = pd.concat([df_posts, pd.DataFrame(posts_data)], ignore_index=True)

    except Exception as e:
        print(f"Error processing post {i}: {str(e)}")

# Sau khi lặp xong, có thể in hoặc lưu DataFrame
print(df_posts.head())

['\\:rb7\\:', '\\:rb8\\:', '\\:rb9\\:', '\\:rba\\:']
[{'time': None, 'num_comment': 'Thích\nBình luận\nChia sẻ', 'like_post': 'Thích, Bày tỏ cảm xúc, Viết bình luận, Gửi nội dung này cho bạn bè hoặc đăng lên trang cá nhân của bạn.', 'time_post': None, 'content_post': 'Bạn làm rơi AirPods nhắn #utcshop tư vấn nha\\nutc có bán lẻ tai, lẻ dock AirPods :\\nchỉ làm hàng zin nguyên bản Apple\\nbao check chính hãng trọn đời… Xem thêm'}]
[{'time': None, 'num_comment': 'Thích\nBình luận\nChia sẻ', 'like_post': 'Thích, Bày tỏ cảm xúc, Viết bình luận, Gửi nội dung này cho bạn bè hoặc đăng lên trang cá nhân của bạn.', 'time_post': None, 'content_post': 'Bạn làm rơi AirPods nhắn #utcshop tư vấn nha\\nutc có bán lẻ tai, lẻ dock AirPods :\\nchỉ làm hàng zin nguyên bản Apple\\nbao check chính hãng trọn đời… Xem thêm'}, {'time': None, 'num_comment': 'Thích\nBình luận\nChia sẻ', 'like_post': 'Thích, Bày tỏ cảm xúc, Viết bình luận, Gửi nội dung này cho bạn bè hoặc đăng lên trang cá nhân của bạn.', 'time_

KeyboardInterrupt: 

# Lấy id shop và id comment của từng post

In [9]:
df_posts = pd.DataFrame(columns=['id_shop', 'id_post'])

for i in range(1, 271):  # Lặp qua 270 posts
    try:
        num_link = None
        num_link_selector = [
            f"/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div[2]/div/div/div/div/div/div[{i}]/div/div/div/div/div/div/div/div/div/div/div[13]/div/div/div[2]/div/div[2]/div/div[1]/span/h3/div/span[1]/a",
            f"/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div[2]/div/div/div/div/div/div[{i}]/div/div/div/div/div/div/div/div/div/div/div[13]/div/div/div[2]/div/div[2]/div/div[1]/span/h3/div/span/a"
        ]

        for selector in num_link_selector:
            try:
                num_link = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, selector)))
                if num_link:
                    break
            except:
                continue
        
        if not num_link:
            print(f"Không tìm thấy link cho post {i}")
            continue

        driver.execute_script("arguments[0].scrollIntoView(true);", num_link)
        new_url = num_link.get_attribute('href')
        time.sleep(10)  # Cân nhắc việc thay thế bằng điều kiện chờ cụ thể thay vì sleep
        print(new_url)

        id_shop = id_post = None
        match = re.search(r'/groups/(\d+)/\?.*multi_permalinks=(\d+)', new_url)
        if match:
            # Chuyển đổi giá trị sang số nguyên
            id_shop = int(match.group(1))
            id_post = int(match.group(2))

        # Đảm bảo giá trị đưa vào DataFrame là số nguyên
        new_row = pd.DataFrame({'id_shop': [id_shop], 'id_post': [id_post]})
        df_posts = pd.concat([df_posts, new_row], ignore_index=True)
        print(f"ID Shop: {id_shop}, ID Post: {id_post}")

    except Exception as e:
        print(f"Lỗi xảy ra: {e}. Bỏ qua.")

driver.quit()

df_posts.to_csv('links_facebook_posts.csv', index=False, encoding='utf-8-sig')

https://www.facebook.com/groups/2882757985295673/?hoisted_section_header_type=recently_seen&multi_permalinks=2944397959131675&__cft__[0]=AZXd0TOj_sL96tpsYtdVTKZtJPWEWWIMgvnGigsYq_g9ADOK4_EKoeR39_gI_8V1If-7rnnjYAOJ_v3FsshqPXf4MhYHi0MPTTiJuBvbQJmXVrPR01iUkJlQJe1GLxPa1VowbwGQnq6jikzdrN-AbwX1JSvKmRhB_GXd16xmtp2eeK-yraYa1Fu02ZeOz18Lwe0&__tn__=-UC%2CP-R
ID Shop: 2882757985295673, ID Post: 2944397959131675
https://www.facebook.com/groups/2882757985295673/?hoisted_section_header_type=recently_seen&multi_permalinks=3379912685580198&__cft__[0]=AZWG9VrohH5Knq1pGjaL6L2TAXYWJH58pWUpykuThgitrsEg4ksrs-vDe0G_zPtZwiymyARWowmHnWbM9sx-hvrWl3f4bcXLSh8GQ6GaAu9nhdxvvC0EbujGusK3E8kkrR-NsiB1SuwaI0xngQhurzdDa4YhFGJL3f3uTQZJrqOCIQK4DmVxPeztQpfbTMSkz6k&__tn__=-UC%2CP-R
ID Shop: 2882757985295673, ID Post: 3379912685580198
https://www.facebook.com/groups/2071636162850853/?hoisted_section_header_type=recently_seen&multi_permalinks=7739693986045014&__cft__[0]=AZXeypx4s_t6SednyImE100gVHKPg6QtIHOelroKZdyhU7DP2ajNnO0E4

# Lấy comment của từng post

In [12]:
df = pd.read_csv(r"D:\projects\NCKH\Data Crawl\Trang MXH\Facebook\links_facebook_posts.csv")
id_shop = df['id_shop'].to_list()
id_post = df['id_post'].to_list()

In [13]:
def safe_find_text(driver, selector, by=By.CSS_SELECTOR):
    try:
        element = driver.find_element(by, selector)
        return element.text
    except NoSuchElementException:
        return None

def safe_get_attribute(driver, selector, attribute, by=By.CSS_SELECTOR):
    try:
        element = driver.find_element(by, selector)
        return element.get_attribute(attribute)
    except NoSuchElementException:
        return None

def click_until_not_present(driver, xpath):
    while True:
        try:
            comment = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, xpath)))
            comment.click()
            time.sleep(5)
        except TimeoutException:
            break
    
for j, k in zip(id_shop, id_post):
    try:
        url = f"https://www.facebook.com/groups/{j}/posts/{k}/"
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[-1])
        driver.get(url)

        current_date = datetime.now().strftime("%Y-%m-%d")
        print(current_date)

        # Sử dụng hàm để click vào "Xem thêm" cho đến khi nó không còn
        button_comment_xemthem_xpath = "/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div[2]/div/div/div[4]/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div[13]/div/div/div[4]/div/div/div[2]/div[3]/div[4]/div[1]/div/div[2]/span/span"
        click_until_not_present(driver, button_comment_xemthem_xpath)

        # Lấy các thông tin cần thiết

        # # Lấy thời gian bài Post
        # # Sửa CSS selector với escape đúng cách cho ':'
        # time_hover_selector = "#\\:r1e1\\: > span:nth-child(2) > span"
        # time_selector = "#\\:r1e1\\: > span:nth-child(2) > span > a > span > span > span"

        # # Khởi tạo ActionChains
        # action = ActionChains(driver)

        # # Thực hiện hover
        # hover_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, time_hover_selector)))
        # action.move_to_element(hover_element).perform()

        # # Đợi để thông tin thời gian hiển thị
        # WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, time_selector)))

        # # Lấy và in ra thông tin thời gian
        # time_element = driver.find_element(By.CSS_SELECTOR, time_selector)
        # print(time_element.text)
        
        # Lấy sô lượt comment
        num_comment = safe_find_text(driver, "/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div[2]/div/div/div[4]/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div[13]/div/div/div[4]/div/div/div[1]/div/div[1]/div/div[2]/div[2]/span/div/span", By.XPATH)
        print(num_comment)
        
        # Lấy nội dung bài post
        content_selector = [
                    "/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div[2]/div/div/div[4]/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div[13]/div/div/div[3]/div[1]/div/span",
                    "/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div[3]/div/div/div[4]/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div[13]/div/div/div[3]/div/div/div/div/span/div/div"
                    ]
        content = next((safe_find_text(driver, selector, By.XPATH) for selector in content_selector if safe_find_text(driver, selector, By.XPATH)), None)
        print(content)
        
        # Lấy số lượt like
        likes = safe_get_attribute(driver, "/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div[2]/div/div/div[4]/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div[13]/div/div/div[4]/div/div/div[1]/div/div[1]/div/div[1]/span/span", "aria-label", By.XPATH)
        print(likes)


        # Lấy thông tin comment

        # # Vào trong phần comment
        # post_comment = f"#mount_0_0_mX > div > div:nth-child(1) > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div:nth-child(13) > div > div > div:nth-child(4) > div > div > div"
        # # Click vào xem tất cả comment
        # els = driver.find_elements(By.CSS_SELECTOR, "div > div:nth-child(1) > div > div:nth-child(5) > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > span")
        # els[0].click()
        # sleep(1)
        # els = driver.find_elements(By.CSS_SELECTOR, "div > div > div:nth-child(1) > div > div:nth-child(5) > div > div > div > div > div:nth-child(3) > div > div > div > div > div > div > div > div > div > div > div > div > div:nth-child(3) > div > div > div:nth-child(1) > span")
        # els[0].click()
        # sleep(5)

        # # Xem tất cả comment
        # while True:
        #     chaps = driver.find_elements(By.CSS_SELECTOR, "#div> div > div:nth-child(1) > div > div:nth-child(5) > div > div > div.x9f619.x1n2onr6.x1ja2u2z > div > div:nth-child(2) > div > div > div > div > div > div > div > div.xb57i2i.x1q594ok.x5lxg6s.x78zum5.xdt5ytf.x6ikm8r.x1ja2u2z.x1pq812k.x1rohswg.xfk6m8.x1yqm8si.xjx87ck.xx8ngbg.xwo3gff.x1n2onr6.x1oyok0e.x1odjw0f.x1iyjqo2.xy5w88m > div.x78zum5.xdt5ytf.x1iyjqo2.x1n2onr6.xaci4zi.x129vozr > div > div > div.xwya9rg.x11i5rnm.x1e56ztr.x1mh8g0r.xh8yej3 > div > div.x1pi30zi.x1swvt13.x1n2onr6 > div.x1gslohp > div:nth-child(4) > div > div > div > div:nth-child(2) > div > div > div > div > div:nth-child(2) > div > div.x78zum5.x1iyjqo2.x21xpn4.x1n2onr6 > div.x1i10hfl.xjbqb8w.xjqpnuy.xa49m3k.xqeqjp1.x2hbi6w.x13fuv20.xu3j5b3.x1q0q8m5.x26u7qi.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xdl72j9.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x2lwn1j.xeuugli.xexx8yu.x18d9i69.xkhd6sd.x1n2onr6.x16tdsg8.x1hl2dhg.xggy1nq.x1ja2u2z.x1t137rt.x1o1ewxj.x3x9cwd.x1e5q0jg.x13rtm0m.x3nfvp2.x1q0g3np.x87ps6o.x1a2a7pz.x6s0dn4.xi81zsa.x1iyjqo2.xs83m0k.xsyo7zv.xt0b8zv > span.x78zum5.x1w0mnb.xeuugli > span")
        #     chaps += driver.find_elements(By.CSS_SELECTOR, "div > div:nth-child(2) > div > div > div > span > span")
        #     chapters = []

        #     for chap in chaps:
        #         # try except -> lỗi kệ nó :)) cứ chạy là được
        #         try:
        #             if "Xem các phản hồi trước" in chap.text:
        #                 chap.click()
        #                 chapters.append(chap)
        #         except Exception as e:
        #             pass
        #     sleep(1)
        #     if len(chapters) == 0:
        #         break
        # # Lấy comment
        # comment_box_selector = "body > div > div > div:nth-child(1) > div > div:nth-child(5) > div > div > div > div > div:nth-child(2) > div > div > div > div > div > div > div:nth-child(2) > div:nth-child(2) > div > div > div:nth-child(2) > div > div:nth-child(2) > div:nth-child(3)"
        # comment_box = driver.find_element(By.CSS_SELECTOR,comment_box_selector)
        # comment_selector = "div > div > div > div > div:nth-child(1) > div > div > div > div > div > div:nth-child(2) > span"
        # comments = comment_box.find_elements(By.CSS_SELECTOR,comment_selector)
        # print(len(comments))
        # for comment in comments:
        #     print(comment.text)


        # Xử lý và lưu thông tin
        product_info = {
            'Platform': 'Facebook',
            'URL': url,
            'Date Crawled': current_date,
            # 'Time': post_time,
            'Num_Comment': num_comment,
            'Content': content,
            'Num_Likes': likes
        }
        
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        
    except Exception as e:
        print(f"Không thể xử lý URL {url}: {e}")
        if len(driver.window_handles) > 1:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

driver.quit()

2024-03-24
None
None
None
2024-03-24


KeyboardInterrupt: 

# Chọn vào những bài viết mới

In [28]:
# Xác định CSS selector cho button và post mới
button_new_selector = f"{class_list[0]} > div > div:nth-child(1) > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div:nth-child(2) > div:nth-child(2) > div > div > div > div > div > div > div > div > div > span > div > div > div > div:nth-child(2) > div > svg"
posts_new_selector = f"{class_list[0]} > div > div:nth-child(1) > div > div > div > div > div:nth-child(2) > div > div > div > div > div > div > div > div > div > div > div > div > div > div:nth-child(3) > div > div > div:nth-child(1) > span"

# Tìm và click vào button mới
button_new = driver.find_element(By.CSS_SELECTOR, button_new_selector)
button_new.click()

# Tìm và click vào post mới
posts_new = driver.find_element(By.CSS_SELECTOR, posts_new_selector)
posts_new.click()

In [ ]:
def convert_time_post_to_actual_datetime(time_post_text):
    match = re.search(r'(\d+)\s*(phút|giờ|ngày) trước', time_post_text)
    if not match:
        return "Không xác định được thời gian"
    
    quantity = int(match.group(1))
    unit = match.group(2)
    
    current_time = datetime.now()
    
    if unit == "phút":
        post_time = current_time - timedelta(minutes=quantity)
    elif unit == "giờ":
        post_time = current_time - timedelta(hours=quantity)
    elif unit == "ngày":
        post_time = current_time - timedelta(days=quantity)
    else:
        return "Đơn vị thời gian không hợp lệ"
    
    return post_time.strftime("%Y-%m-%d %H:%M:%S")


In [61]:
num_comment

'34 bình luận'

In [47]:
# Assuming 'driver' has been already initialized
action = ActionChains(driver)

# Modify the selector to match your needs
show_comment_selector = "#mount_0_0_cu > div > div:nth-child(1) > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div:nth-child(2) > div:nth-child(2) > div:nth-child(2) > div > div > div > div > div > div > div > div > div > div:nth-child(8) > div > div > div:nth-child(4) > div > div > div > div > div > div > span > span"
try:
    # Wait for the comment button to be clickable
    comment_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, show_comment_selector))
    )
    sleep(1)  # Just to ensure the page is ready
    action.click(comment_button).perform()
except NoSuchElementException:
    print("Element not found")
except TimeoutException:
    print("Element not clickable")

In [56]:
# Bấm vào nút xem thêm
see_more_button_selector = "#mount_0_0_9n > div > div:nth-child(1) > div > div.x9f619.x1n2onr6.x1ja2u2z > div > div > div.x78zum5.xdt5ytf.x1t2pt76.x1n2onr6.x1ja2u2z.x10cihs4 > div.x78zum5.xdt5ytf.x1iyjqo2.x1us19tq > div > div.x9f619.x2lah0s.x1n2onr6.x78zum5.x1iyjqo2.x1t2pt76.x1lspesw > div > div > div.x78zum5.xdt5ytf.x1iyjqo2 > div > div > div.x6s0dn4.x78zum5.xdt5ytf.xwib8y2.xh8yej3 > div > div > div > div:nth-child(2) > div:nth-child(2) > div:nth-child(2) > div > div > div > div > div > div > div > div > div > div:nth-child(8) > div > div > div:nth-child(4) > div > div > div.xzueoph > div.xdj266r.xktsk01.xat24cr.x1d52u69 > div > div.x1i10hfl.xjbqb8w.xjqpnuy.xa49m3k.xqeqjp1.x2hbi6w.x13fuv20.xu3j5b3.x1q0q8m5.x26u7qi.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xdl72j9.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x2lwn1j.xeuugli.xexx8yu.x18d9i69.xkhd6sd.x1n2onr6.x16tdsg8.x1hl2dhg.xggy1nq.x1ja2u2z.x1t137rt.x1o1ewxj.x3x9cwd.x1e5q0jg.x13rtm0m.x3nfvp2.x1q0g3np.x87ps6o.x1a2a7pz.x6s0dn4.xi81zsa.x1iyjqo2.xs83m0k.xsyo7zv.xt0b8zv > span > span"
els = driver.find_elements(By.CSS_SELECTOR,see_more_button_selector)
if len(els) > 0:
    el = els[-1]
    el.click()

# Lấy tổng quan của bài viết

In [59]:
# Lấy số comment và rating
num_comment = "#\:r2f\: > span"
num_like = "#mount_0_0_9n > div > div:nth-child(1) > div > div:nth-child(5) > div > div > div.x9f619.x1n2onr6.x1ja2u2z > div > div.x1uvtmcs.x4k7w5x.x1h91t0o.x1beo9mf.xaigb6o.x12ejxvf.x3igimt.xarpa2k.xedcshv.x1lytzrv.x1t2pt76.x7ja8zs.x1n2onr6.x1qrby5j.x1jfb8zj > div > div > div > div > div > div > div.xb57i2i.x1q594ok.x5lxg6s.x78zum5.xdt5ytf.x6ikm8r.x1ja2u2z.x1pq812k.x1rohswg.xfk6m8.x1yqm8si.xjx87ck.xx8ngbg.xwo3gff.x1n2onr6.x1oyok0e.x1odjw0f.x1iyjqo2.xy5w88m > div.x78zum5.xdt5ytf.x1iyjqo2.x1n2onr6.xaci4zi.x129vozr > div > div > div.xwya9rg.x11i5rnm.x1e56ztr.x1mh8g0r.xh8yej3 > div > div:nth-child(1) > div > div.x1n2onr6 > div > div.x6s0dn4.x78zum5.x1iyjqo2.x6ikm8r.x10wlt62 > div > span > div > span.xt0b8zv.x2bj2ny.xrbpyxo.xl423tq > span > span"
title_content = "#\:r85\: > div > div > span > div > div"

All_post = driver.find_elements(By.CSS_SELECTOR, num_comment)
All_post += driver.find_elements(By.CSS_SELECTOR, num_like)
All_post += driver.find_elements(By.CSS_SELECTOR, title_content)
result = []
for i in range(len(All_post)):
    if All_post[i].text != "":
        result.append(All_post[i].text)
print(result)

['36 bình luận', '36 bình luận', '9', '15pro max titan tự nhiên vn/a vừa bóc hộp hôm qua do ko dùng quen bán lại cho a e nào thiện chí 29trxx kèm củ sạc ốp đã dán cl khu vực bắc giang bắc ninh.']


In [60]:
# Sửa CSS selector với escape đúng cách cho ':'
time_hover_selector = "#\\:r1e1\\: > span:nth-child(2) > span"
time_selector = "#\\:r1e1\\: > span:nth-child(2) > span > a > span > span > span"

# Khởi tạo ActionChains
action = ActionChains(driver)

# Thực hiện hover
hover_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, time_hover_selector)))
action.move_to_element(hover_element).perform()

# Đợi để thông tin thời gian hiển thị
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, time_selector)))

# Lấy và in ra thông tin thời gian
time_element = driver.find_element(By.CSS_SELECTOR, time_selector)
print(time_element.text)

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00BD8D33+51395]
	(No symbol) [0x00B45F91]
	(No symbol) [0x009FE13A]
	(No symbol) [0x00A362BB]
	(No symbol) [0x00A363EB]
	(No symbol) [0x00A6C162]
	(No symbol) [0x00A53ED4]
	(No symbol) [0x00A6A570]
	(No symbol) [0x00A53C26]
	(No symbol) [0x00A2C629]
	(No symbol) [0x00A2D40D]
	GetHandleVerifier [0x00F564F3+3711107]
	GetHandleVerifier [0x00F958DA+3970154]
	GetHandleVerifier [0x00F90BC8+3950424]
	GetHandleVerifier [0x00C89D39+776393]
	(No symbol) [0x00B51764]
	(No symbol) [0x00B4C648]
	(No symbol) [0x00B4C7F9]
	(No symbol) [0x00B3DE20]
	BaseThreadInitThunk [0x761C7BA9+25]
	RtlInitializeExceptionChain [0x77D7BD2B+107]
	RtlClearBits [0x77D7BCAF+191]


# Lấy các comment bài viết

In [41]:
# Click vào xem tất cả comment
els = driver.find_elements(By.CSS_SELECTOR, "div > div:nth-child(1) > div > div:nth-child(5) > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > div > span")
els[0].click()
sleep(1)
els = driver.find_elements(By.CSS_SELECTOR, "div > div > div:nth-child(1) > div > div:nth-child(5) > div > div > div > div > div:nth-child(3) > div > div > div > div > div > div > div > div > div > div > div > div > div:nth-child(3) > div > div > div:nth-child(1) > span")
els[0].click()
sleep(5)

# Xem tất cả comment
while True:
    chaps = driver.find_elements(By.CSS_SELECTOR, "#div> div > div:nth-child(1) > div > div:nth-child(5) > div > div > div.x9f619.x1n2onr6.x1ja2u2z > div > div:nth-child(2) > div > div > div > div > div > div > div > div.xb57i2i.x1q594ok.x5lxg6s.x78zum5.xdt5ytf.x6ikm8r.x1ja2u2z.x1pq812k.x1rohswg.xfk6m8.x1yqm8si.xjx87ck.xx8ngbg.xwo3gff.x1n2onr6.x1oyok0e.x1odjw0f.x1iyjqo2.xy5w88m > div.x78zum5.xdt5ytf.x1iyjqo2.x1n2onr6.xaci4zi.x129vozr > div > div > div.xwya9rg.x11i5rnm.x1e56ztr.x1mh8g0r.xh8yej3 > div > div.x1pi30zi.x1swvt13.x1n2onr6 > div.x1gslohp > div:nth-child(4) > div > div > div > div:nth-child(2) > div > div > div > div > div:nth-child(2) > div > div.x78zum5.x1iyjqo2.x21xpn4.x1n2onr6 > div.x1i10hfl.xjbqb8w.xjqpnuy.xa49m3k.xqeqjp1.x2hbi6w.x13fuv20.xu3j5b3.x1q0q8m5.x26u7qi.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xdl72j9.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x2lwn1j.xeuugli.xexx8yu.x18d9i69.xkhd6sd.x1n2onr6.x16tdsg8.x1hl2dhg.xggy1nq.x1ja2u2z.x1t137rt.x1o1ewxj.x3x9cwd.x1e5q0jg.x13rtm0m.x3nfvp2.x1q0g3np.x87ps6o.x1a2a7pz.x6s0dn4.xi81zsa.x1iyjqo2.xs83m0k.xsyo7zv.xt0b8zv > span.x78zum5.x1w0mnb.xeuugli > span")
    chaps += driver.find_elements(By.CSS_SELECTOR, "div > div:nth-child(2) > div > div > div > span > span")
    chapters = []

    for chap in chaps:
        # try except -> lỗi kệ nó :)) cứ chạy là được
        try:
            if "Xem các phản hồi trước" in chap.text:
                chap.click()
                chapters.append(chap)
        except Exception as e:
            pass
    sleep(1)
    if len(chapters) == 0:
        break
# Lấy comment
comment_box_selector = "body > div > div > div:nth-child(1) > div > div:nth-child(5) > div > div > div > div > div:nth-child(2) > div > div > div > div > div > div > div:nth-child(2) > div:nth-child(2) > div > div > div:nth-child(2) > div > div:nth-child(2) > div:nth-child(3)"
comment_box = driver.find_element(By.CSS_SELECTOR,comment_box_selector)
comment_selector = "div > div > div > div > div:nth-child(1) > div > div > div > div > div > div:nth-child(2) > span"
comments = comment_box.find_elements(By.CSS_SELECTOR,comment_selector)
print(len(comments))
for comment in comments:
    print(comment.text)

24
R chắc ổng đọc cfs ni hok bíc bà đâu =)))
Nhung Trương biết rút kinh nghiệm chưa?
Ann Thư rút kịp kh ?
Nhung Trương thu hồi nma k hiệu quả
Ann Thư thu chi rồi cũng zậy mà
Nhung Trương tốt nhất là người dùng hiện tại k có mặt trên fb :))
Mỹ Duyên có hay vậy k?
Dung Phan hay đi stalk và cái kết =))
Dương Hồng Nga cứu bạn
Lê Trang hết cíu đc òi kkk
e hỏi a Hà Phước Thịnh ni nè e, a ni kinh nghiệm
Nghỉ học đi c
Hoang Le Thanh Tam cẩn thận đó
Thiên Linh Lãnh Hàn nhớ bữa stalk tới đoạn "thả icon tớ sẽ nói những gì về cậu" cái nhấn like
Ngân Đoàn Hồng Minh đứa nào khai mau
Bùi Thị Trà My chắc tui đó bà
NguyễnQ. Hương m đúng ko
Thảo Nhi Bài học rút ra: Đi stalk thì cầm đt cho chắc tay
Đoàn Nguyễn Hồng Minh stalk ngta đừng run tay nha :)))
Hồ Khả Thuỳ Dương hong dám stalk nựa
Thanh Truc stalk hạn chế run tay nghe b
Hoàng Ngân tui stalk làm chi họ buýt đc
Thanh Truc ghê sư phụ có khác
=))) đăng lên z lát cha nụi đó dô cho bí quyết đó
